<a href="https://colab.research.google.com/github/100363504/MasterDL/blob/main/Topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En este código vamos a procesar un conjunto de datos que contiene artículos educativos para conseguir clasificarlos por tópicos y que sea más fácil el manejo de esta amplia base de datos.
Se va a modelar con BERTopic.

In [ ]:
# Instalamos e importamos los paquetes necesarios.
!pip install bertopic

In [2]:
from bertopic import BERTopic
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import json
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [3]:
# Los datos se encuentran almacenados en Drive, por lo que conectamos nuestro entorno a la carpeta donde se encuentran estos datos.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
file_path = "/content/drive/My Drive/Master UAH/Paralelizacion/data_part1.1.1.1.json"
file_path2 = "/content/drive/My Drive/Master UAH/Paralelizacion/arxiv-metadata-oai-snapshot.json"



In [5]:
# Leemos el archivo JSON donde se encuentran los artículos y obtenemos una lista de documentos 'abstract'
docs = []
with open(file_path, "r", encoding="utf-8") as file:
    for line in file:
        data = json.loads(line.strip())
        abstract_text = data['abstract']
        docs.append(abstract_text)

print(docs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Primero hacemos un preprocesamiento simple de los datos, para ver la diferencia entre un preprocesamiento más sencillo y uno más complejo.

In [14]:
# Tokenización
docs_tokenized = [nltk.word_tokenize(doc) for doc in docs]

# Normalización de texto: Convertir a minúsculas y eliminar caracteres especiales
docs_preprocessed = [[re.sub(r'[^a-zA-Z]', '', word.lower()) for word in doc] for doc in docs_tokenized]

docs_preprocessed = [' '.join(doc) if isinstance(doc, list) else str(doc) for doc in docs_preprocessed]


Este es el proprocesamiento más complejo. Se usan técnicas como la tokenización, la eliminación de palabras vacías, la normalización de texto y la lematización.

In [ ]:
# Tokenización
docs_tokenized = [nltk.word_tokenize(doc) for doc in docs]

# Eliminación de palabras vacías (stop words)
stop_words = set(stopwords.words('english'))
docs_filtered = [[word for word in doc if word.lower() not in stop_words] for doc in docs_tokenized]

# Normalización de texto: Convertir a minúsculas y eliminar caracteres especiales
docs_normalized = [[re.sub(r'[^a-zA-Z0-9]', '', word.lower()) for word in doc] for doc in docs_filtered]

# Lematización
lemmatizer = WordNetLemmatizer()
docs_lemmatized = [[lemmatizer.lemmatize(word) for word in doc] for doc in docs_normalized]

# Unir tokens preprocesados en documentos nuevamente
docs_preprocessed = [' '.join(doc) for doc in docs_lemmatized]



In [ ]:
# Por último, creamos y ajustamos el modelo BERTopic en los documentos preprocesados


topic_model = BERTopic()
topics, _ = topic_model.fit_transform(docs_preprocessed)

In [ ]:
topic_model.get_topic_info()